## Imports

In [2]:
import time

import numpy as np
import cv2 as cv
import os
import glob
import matplotlib.pyplot as plt
from cv2 import cvtColor
from numpy.random import uniform
import pdb
from tqdm import tqdm

import utils
from utils import show_image_matplot,show_image_cv

In [3]:
game_tables = utils.extract_game(utils.get_image_paths("antrenare",1))

100%|██████████| 50/50 [00:14<00:00,  3.46it/s]


In [4]:
relevant_game_tables = utils.extract_relevant_game(game_tables)

100%|██████████| 50/50 [00:00<00:00, 96.10it/s]


In [5]:
delimited_tables = utils.draw_lines(relevant_game_tables)

100%|██████████| 50/50 [00:00<00:00, 921.42it/s]


In [6]:
# utils.store_template_numbers1(10)
# utils.store_template_numbers2(8)

In [7]:
# utils.store_binary_templates(100,"./templates/sample1","./templates_binary/sample1")
# utils.store_binary_templates(100)

In [8]:
# new_templates = crop_and_store_templates()

In [9]:
#utils.rewrite_all()

In [17]:
from utils import get_mathable_pieces_numbers, get_lines_coords

# def classify_number(patch,templates_path = "./templates_cropped"):
#     max_corr = -np.inf
#     chosen_number = -1
#     numbers = get_mathable_pieces_numbers()
#
#
#     for i in range(10,len(numbers)):
#
#         number = numbers[i]
#         template = cv.imread(templates_path+ "/" + str(number) + ".jpg")
#         template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
#
#         corr = cv.matchTemplate(patch,template, cv.TM_CCOEFF_NORMED)
#         corr=np.max(corr)
#
#         if corr > max_corr:
#             max_corr = corr
#             chosen_number = number
#
#     if max_corr >= 0.8:
#         return chosen_number
#
#     max_corr = -np.inf
#     chosen_number = -1
#
#     for i in range(0,10):
#
#         number = numbers[i]
#         template = cv.imread(templates_path+ "/" + str(number) + ".jpg")
#         template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
#
#         corr = cv.matchTemplate(patch,template, cv.TM_CCOEFF_NORMED)
#         corr=np.max(corr)
#
#         if corr > max_corr:
#             max_corr = corr
#             chosen_number = number
#
#
#     return chosen_number

def classify_number(patch,templates_path = "./templates_cropped"):

    maxi = -np.inf
    chosen_number = -1
    numbers = get_mathable_pieces_numbers()

    num_whites_patch = np.sum(patch == 255)

    for number in numbers:
        template = cv.imread(templates_path+ "/" + str(number) + ".jpg")
        template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)

        num_whites_template = np.sum(template == 255)

        if num_whites_template * 1.95 < num_whites_patch:
            continue

        corr = cv.matchTemplate(patch,template, cv.TM_CCOEFF_NORMED)
        corr=np.max(corr)

        if corr > maxi:
            maxi = corr
            chosen_number = number


    return chosen_number

def match_numbers(binary_image, offset):
    lines_vertical,lines_horizontal = get_lines_coords()

    coords={}

    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):

            y_min = lines_vertical[j][0][0] + offset
            y_max = lines_vertical[j + 1][1][0] - offset
            x_min = lines_horizontal[i][0][1] + offset
            x_max = lines_horizontal[i + 1][1][1] - offset

            patch = binary_image[x_min:x_max,y_min:y_max].copy()

            if (i==6 or i==7) and (j==6 or j==7):
                continue

            num_white_pixels = np.sum(binary_image[x_min+30:x_max-30,y_min+30:y_max-30].copy() == 255)

            if num_white_pixels > 0:
                predicted_number = classify_number(patch)
                coords[(i,j)] = predicted_number

    return coords

def get_patches(binary_image,offset):
    lines_vertical,lines_horizontal = get_lines_coords()

    patches = []

    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):

            y_min = lines_vertical[j][0][0] + offset
            y_max = lines_vertical[j + 1][1][0] - offset
            x_min = lines_horizontal[i][0][1] + offset
            x_max = lines_horizontal[i + 1][1][1] - offset

            patch = binary_image[x_min:x_max,y_min:y_max].copy()

            patches.append(patch)

    return patches

In [11]:
def move_train_data(train_number,dest_path = "./train"):
    img_prefix = str(train_number)+ "_"

    for i in range(9):
        img_name = img_prefix + "0" +str(i+1) + ".jpg"
        img_path = "./antrenare/" + img_name
        img = cv.imread(img_path)
        cv.imwrite(dest_path +"/" + img_name, img)

    for i in range(9,50):
        img_path = "./antrenare/" + img_prefix + str(i+1) + ".jpg"
        img_name = img_prefix +str(i+1) + ".jpg"
        img = cv.imread(img_path)
        cv.imwrite(dest_path +"/" + img_name, img)

# for i in range(4):
#     move_train_data(i+1)



In [24]:

def solve(train_number,src_path = "./antrenare",output_dest_path = "./343_Gabroveanu_Razvan"):

    game_tables = utils.extract_game(utils.get_image_paths(src_path,train_number))
    relevant_game_tables = utils.extract_relevant_game(game_tables)

    visited = np.full((14,14),-1)

    visited[6,6] = 1
    visited[6,7] = 2
    visited[7,6] = 3
    visited[7,7] = 4

    output_file_prefix = "/" + str(train_number) + "_"

    for i in tqdm(range(len(relevant_game_tables))):

        table = relevant_game_tables[i]
        table = cv.cvtColor(table, cv.COLOR_BGR2GRAY)
        _,table_binary = cv.threshold(table,65,255,cv.THRESH_BINARY_INV)

        result_dict = match_numbers(table_binary,8)

        for indexes,predicted_number in result_dict.items():
            row,col = indexes

            if visited[row,col] == -1:
                continue

            output = str(row+1) + chr(65 + col)+ " " + str(predicted_number)
            output_file_path = output_dest_path + output_file_prefix

            if i in range(0,9):
                output_file_path = output_file_path + "0"

            output_file_path = output_file_path + str(i+1) + ".txt"

            with open(output_file_path,"w") as file:
                file.write(output)

            visited[row,col] = predicted_number

    with open(output_dest_path + "/" + str(train_number) + "_scores.txt","w") as file:
        pass

    with open(output_dest_path + "/" + str(train_number) + "_turns.txt","w") as file:
        pass

for i in range(4):
    solve(i+1)



100%|██████████| 50/50 [00:17<00:00,  2.90it/s]


In [13]:

game_tables2 = utils.extract_game(utils.get_image_paths("antrenare",2))
relevant_game_tables2 = utils.extract_relevant_game(game_tables2)

100%|██████████| 50/50 [00:00<00:00, 117.10it/s]


In [14]:

sample = relevant_game_tables2[0]
sample = cv.cvtColor(sample,cv.COLOR_BGR2GRAY)
_,sample = cv.threshold(sample,65,255,cv.THRESH_BINARY_INV)

show_image_cv("",sample,fx = 0.4,fy = 0.4)

coords = match_numbers(sample, offset = 8)
patches = get_patches(sample, offset = 8)

In [15]:
print(coords)

{(6, 8): 8}


In [16]:
for key,value in coords.items():
    index = key[0] * 14 + key[1]
    title = str(key[0]) + "," + str(key[1]) + " " + str(value)
    # show_image_cv(title,patches[index],fx = 1,fy = 1)

    three = cv.imread("templates_cropped/3.jpg")
    eight = cv.imread("templates_cropped/8.jpg")


    three = cv.cvtColor(three,cv.COLOR_BGR2GRAY)
    eight= cv.cvtColor(eight,cv.COLOR_BGR2GRAY)

    # _, three = cv.threshold(three, 128, 255, cv.THRESH_BINARY)
    # _, eight = cv.threshold(eight, 128, 255, cv.THRESH_BINARY)

    w3 = np.sum(three == 255)
    wp = np.sum(patches[index] == 255)

    corr1 = np.max(cv.matchTemplate(patches[index],three,cv.TM_CCOEFF_NORMED))
    corr2 = np.max(cv.matchTemplate(patches[index],eight,cv.TM_CCOEFF_NORMED))
    cv.imwrite("temp.jpg",patches[index])
    print(corr1,corr2)
    print(three.shape)
    print(eight.shape)
    print(patches[index].shape)
    print(w3,wp)
    print(classify_number(patches[index]))

0.8862497 0.7159185
(72, 45)
(71, 45)
(134, 134)
805 1660
8


In [171]:

class Solver():

    def __init__(self,data_set_id):

        self.data_set_id = data_set_id

        self.table_configuration = np.full((14,14),-1)
        self.table_configuration[6,6] = 1
        self.table_configuration[6,7] = 2
        self.table_configuration[7,6] = 3
        self.table_configuration[7,7] = 4

        self.ADD = 1
        self.SUB = 2
        self.MUL = 3
        self.DIV = 4
        self.TIMES_2 = 5
        self.TIMES_3 = 6

        self.table_constraints = np.array([
            [self.TIMES_3,0,0,0,0,0,self.TIMES_3,self.TIMES_3,0,0,0,0,0,self.TIMES_3],
            [0,self.TIMES_2,0,0,self.DIV,0,0,0,0,self.DIV,0,0,self.TIMES_2,0],
            [0,0,self.TIMES_2,0,0,self.SUB,0,0,self.SUB,0,0,self.TIMES_2,0,0],
            [0,0,0,self.TIMES_2,0,0,self.ADD,self.MUL,0,0,self.TIMES_2,0,0,0],
            [0,self.DIV,0,0,self.TIMES_2,0,self.MUL,self.ADD,0,self.TIMES_2,0,0,self.DIV,0],
            [0,0,self.SUB,0,0,0,0,0,0,0,0,self.SUB,0,0],
            [self.TIMES_3,0,0,self.MUL,self.ADD,0,0,0,0,self.MUL,self.ADD,0,0,self.TIMES_3],

            [self.TIMES_3,0,0,self.ADD,self.MUL,0,0,0,0,self.ADD,self.MUL,0,0,self.TIMES_3],
            [0,0,self.SUB,0,0,0,0,0,0,0,0,self.SUB,0,0],
            [0,self.DIV,0,0,self.TIMES_2,0,self.ADD,self.MUL,0,self.TIMES_2,0,0,self.DIV,0],
            [0,0,0,self.TIMES_2,0,0,self.MUL,self.ADD,0,0,self.TIMES_2,0,0,0],
            [0,0,self.TIMES_2,0,0,self.SUB,0,0,self.SUB,0,0,self.TIMES_2,0,0],
            [0,self.TIMES_2,0,0,self.DIV,0,0,0,0,self.DIV,0,0,self.TIMES_2,0],
            [self.TIMES_3,0,0,0,0,0,self.TIMES_3,self.TIMES_3,0,0,0,0,0,self.TIMES_3],

        ])


    def reset(self):
        self.table_configuration = np.full((14,14),-1)
        self.table_configuration[6,6] = 1
        self.table_configuration[6,7] = 2
        self.table_configuration[7,6] = 3
        self.table_configuration[7,7] = 4


    def solve(self,src_path = "./antrenare",output_dest_path = "./343_Gabroveanu_Razvan"):

        self.reset()

        game_tables = utils.extract_game(utils.get_image_paths(src_path,self.data_set_id))
        relevant_game_tables = utils.extract_relevant_game(game_tables)

        output_file_prefix = "/" + str(self.data_set_id) + "_"

        for i in tqdm(range(len(relevant_game_tables))):

            table = relevant_game_tables[i]
            table = cv.cvtColor(table, cv.COLOR_BGR2GRAY)
            _,table_binary = cv.threshold(table,65,255,cv.THRESH_BINARY_INV)

            result_dict = self.match_numbers(table_binary,8)

            for indexes,predicted_number in result_dict.items():
                row,col = indexes

                if self.table_configuration[row,col] != -1:
                    continue

                output = str(row+1) + chr(65 + col)+ " " + str(predicted_number)
                output_file_path = output_dest_path + output_file_prefix

                if i in range(0,9):
                    output_file_path = output_file_path + "0"

                output_file_path = output_file_path + str(i+1) + ".txt"

                with open(output_file_path,"w") as file:
                    file.write(output)

                self.table_configuration[row,col] = predicted_number

        with open(output_dest_path + "/" + str(self.data_set_id) + "_scores.txt","w") as file:
            pass

        with open(output_dest_path + "/" + str(self.data_set_id) + "_turns.txt","w") as file:
            pass


    def match_numbers(self,binary_image, offset):
        lines_vertical,lines_horizontal = get_lines_coords()

        coords={}

        for i in range(len(lines_horizontal)-1):
            for j in range(len(lines_vertical)-1):

                y_min = lines_vertical[j][0][0] + offset
                y_max = lines_vertical[j + 1][1][0] - offset
                x_min = lines_horizontal[i][0][1] + offset
                x_max = lines_horizontal[i + 1][1][1] - offset

                patch = binary_image[x_min:x_max,y_min:y_max].copy()

                if (i==6 or i==7) and (j==6 or j==7):
                    continue

                num_white_pixels = np.sum(binary_image[x_min+30:x_max-30,y_min+30:y_max-30].copy() == 255)

                if num_white_pixels > 0 and self.table_configuration[i,j] != -1:

                    possible_numbers = self.get_possible_values(i,j)

                    predicted_number = self.classify_number(patch,possible_numbers)
                    coords[(i,j)] = predicted_number

        return coords

    def classify_number(self,patch,possible_numbers,templates_path = "./templates_cropped"):

        maxi = -np.inf
        chosen_number = -1
        print(possible_numbers)
        num_whites_patch = np.sum(patch == 255)

        for number in possible_numbers:

            path = templates_path+ "/" + str(number) + ".jpg"

            if not os.path.exists(path):
                continue

            template = cv.imread(path)
            template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)

            num_whites_template = np.sum(template == 255)

            if num_whites_template * 1.95 < num_whites_patch:
                continue

            corr = cv.matchTemplate(patch,template, cv.TM_CCOEFF_NORMED)
            corr=np.max(corr)

            if corr > maxi:
                maxi = corr
                chosen_number = number


        return chosen_number

    def get_neighbour_cells_indexes(self, i, j):
        neighbours = []

        # UP
        if i > 1 and self.table_configuration[i-1,j] != -1 and self.table_configuration[i-2,j] != -1:
            neighbours.append([(i-1,j),(i-2,j)])
        #DOWN
        if i < 14 - 2 and self.table_configuration[i+1,j] != -1 and self.table_configuration[i+2,j] != -1:
            neighbours.append([(i+1,j),(i+2,j)])
        # LEFT
        if j > 1 and self.table_configuration[i,j-1] != -1 and self.table_configuration[i,j-2] != -1:
            neighbours.append([(i,j-1),(i,j-2)])
        # RIGHT
        if j < 14 - 2 and self.table_configuration[i,j+1] != -1 and self.table_configuration[i,j+2] != -1:
            neighbours.append([(i,j+1),(i,j+2)])

        return neighbours


    def get_possible_values(self,i,j):

        neighbours_indexes = self.get_neighbour_cells_indexes(i,j)

        possible_piece_values = []

        current_cell_constraint = self.table_constraints[i,j]

        for pair in neighbours_indexes:
            i1,j1 = pair[0]
            i2,j2 = pair[1]

            neigh1 = self.table_configuration[i1,j1]
            neigh2 = self.table_configuration[i2,j2]

            # Check constraints
            if current_cell_constraint == self.ADD:
                possible_piece_values.append(neigh2 + neigh1)
                continue

            if current_cell_constraint == self.MUL:
                possible_piece_values.append(neigh2 * neigh1)
                continue

            if current_cell_constraint == self.DIV:

                if neigh2 != 0 and neigh1 % neigh2 == 0 :
                    possible_piece_values.append(neigh1//neigh2)

                elif neigh1 != 0 and neigh2 % neigh1 == 0 :
                    possible_piece_values.append(neigh2//neigh1)

                continue

            if current_cell_constraint == self.SUB:
                possible_piece_values.append(abs(neigh2 - neigh1))
                continue

            # No constraints case
            possible_piece_values.append(neigh2 + neigh1)
            possible_piece_values.append(neigh2 * neigh1)
            possible_piece_values.append(abs(neigh2 - neigh1))

            if neigh2 != 0 and neigh1 % neigh2 == 0 :
                possible_piece_values.append(neigh1//neigh2)

            elif neigh1 != 0 and neigh2 % neigh1 == 0 :
                possible_piece_values.append(neigh2//neigh1)

        return list(set(possible_piece_values))


    def calculate_score(self,i,j):

        assert self.table_configuration[i,j] != -1, "A cell must have a value in order to calculate a score."

        neighbours_indexes = self.get_neighbour_cells_indexes(i,j)
        value = self.table_configuration[i,j]
        current_constraint = self.table_constraints[i,j]

        score = 0

        # Just addition score
        if current_constraint == self.ADD:

            for pair in neighbours_indexes:
                i1,j1 = pair[0]
                i2,j2 = pair[1]

                neigh1 = self.table_configuration[i1,j1]
                neigh2 = self.table_configuration[i2,j2]

                score += value if neigh1 + neigh2 == value else 0
            return score

        # Just multiply score
        if current_constraint == self.MUL:

             for pair in neighbours_indexes:
                i1,j1 = pair[0]
                i2,j2 = pair[1]

                neigh1 = self.table_configuration[i1,j1]
                neigh2 = self.table_configuration[i2,j2]

                score += value if neigh1 * neigh2 == value else 0
             return score

        # Just subtraction score
        if current_constraint == self.SUB:

            for pair in neighbours_indexes:
                i1,j1 = pair[0]
                i2,j2 = pair[1]

                neigh1 = self.table_configuration[i1,j1]
                neigh2 = self.table_configuration[i2,j2]

                score += value if abs(neigh1 - neigh2) == value else 0
            return score

        # Just division score
        if current_constraint == self.DIV:

            for pair in neighbours_indexes:
                i1,j1 = pair[0]
                i2,j2 = pair[1]

                neigh1 = self.table_configuration[i1,j1]
                neigh2 = self.table_configuration[i2,j2]

                if neigh2 != 0 and neigh1 % neigh2 == 0:
                    score += value if neigh1//neigh2 == value else 0

                elif neigh1 != 0 and neigh2 % neigh1 == 0:
                    score += value if neigh2//neigh1 == value else 0
            return score


        # No operator constraint, test them all

        for pair in neighbours_indexes:
                i1,j1 = pair[0]
                i2,j2 = pair[1]

                neigh1 = self.table_configuration[i1,j1]
                neigh2 = self.table_configuration[i2,j2]

                if neigh1 + neigh2 == value or neigh1 * neigh2 == value or abs(neigh1 - neigh2) == value:
                    score += value
                    continue

                if (neigh2 != 0 and neigh1 % neigh2 == 0 and neigh1//neigh2 == value) or (neigh1 != 0 and neigh2 % neigh1 == 0 and neigh2//neigh1==value):

                    score += value
                    continue

        # return the score and check for bonus constraints
        return score * 3 if current_constraint == self.TIMES_3 else score * 2 if current_constraint == self.TIMES_2 else score



solver = Solver(2)

i,j = 6,8
solver.table_configuration[i,j] = 3
print(solver.table_configuration)
print(solver.get_possible_values(i,j+1))


[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1  1  2  3 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1  3  4 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]
[6]


 54%|█████▍    | 27/50 [00:07<00:06,  3.51it/s]


KeyboardInterrupt: 

In [172]:
for i in range(4):
    solver = Solver(i+1)
    solver.solve()

100%|██████████| 50/50 [00:00<00:00, 251.12it/s]


In [181]:
img = utils.extract_game(["./antrenare/2_17.jpg"])
img = utils.extract_relevant_game(img)[0]
img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
_,table_binary = cv.threshold(img,65,255,cv.THRESH_BINARY_INV)

show_image_cv("",table_binary,fx=0.3,fy=0.3)


100%|██████████| 1/1 [00:00<00:00, 87.17it/s]
